In [2]:
from __future__ import print_function

In [148]:
import pyLDAvis
import pandas as pd
import numpy as np
import json
# import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import sys
sys.path.append("..")
from collections import Counter

In [14]:
topic_word_df = pd.read_csv('../data/topic_word_matrix.csv', header=None)

In [16]:
# topic word distribution
topic_word = topic_word_df.to_numpy(copy=True)

In [18]:
topic_word.shape

(10, 10000)

In [20]:
with open('../data/doc_topic_matrix.txt', 'r') as f:
    doc_topic_dist = f.read()

In [23]:
d1 = doc_topic_dist.replace('WrappedArray', '')

In [30]:
d2 = d1.replace('(', '')
d3 = d2.replace(')', '')

In [32]:
with open('../data/doc_topic_matrix_parsed.csv', 'w') as f:
    f.write(d3)

In [33]:
doc_top_df = pd.read_csv('../data/doc_topic_matrix_parsed.csv', header=None)

In [38]:
# document topic distribution
doc_top = doc_top_df.to_numpy(copy=True)

In [40]:
doc_top.shape

(115648, 10)

In [41]:
# document lengths
doc_lengths = np.loadtxt('../data/doc_lengths.txt')

In [45]:
len(doc_lengths)

115648

In [48]:
with open('../data/vocabulary.txt', 'r') as f:
    vocab = f.read()

In [59]:
# vocabulary
vocabulary = vocab.split('\n')
vocabulary.pop()

''

In [60]:
len(vocabulary)

10000

In [68]:
with open('../data/tfdata.json') as f:
    tf = f.read()

In [73]:
tf2 = tf.replace('\n', ',')

In [83]:
header = "{\"tfdata\": ["
footer = "]}"

In [84]:
tf3 = "".join([header,tf2])

In [86]:
tf4 = "".join([tf3,footer])

In [131]:
tfjson = json.loads(tf4)

In [181]:
def term_freq(json_doc):
    c = Counter(dict(zip(json_doc['tfdata'][0]['indices'], json_doc['tfdata'][0]['values'])))
    for i in range(1, len(json_doc['tfdata'])):
        c.update(dict(zip(json_doc['tfdata'][i]['indices'], json_doc['tfdata'][i]['values'])))
    out = []
    for k in sorted(c.keys()):
        out.append(c[k])
        if i % 100 == 0:
            print('Iteration: ' + str(i))
    return out

In [182]:
term_freq_corpus = term_freq(tfjson)

In [184]:
len(term_freq_corpus)

10000

In [185]:
prepared_data = pyLDAvis.prepare(topic_word, doc_top, doc_lengths, vocabulary, term_freq_corpus)

/Users/jeremymiller/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [186]:
pyLDAvis.display(prepared_data)

In [187]:
pyLDAvis.save_html(prepared_data, '../html/TopicModeling.html')